## 1. 引言
在上上一节[带你构建MiniGPT](https://golfxiao.blog.csdn.net/article/details/144162768)中详细演示了文本生成的过程，整个过程如下图所示：

![文本生成过程](./img/10-1.png)

上图描绘了使用 GPT 模型进行文本生成的三步。
- 首先，分词器将输入文本转换为一系列的 token ID。
- 其次，模型接收这些 token ID，并生成相应的 logit，这些 logit 是向量，代表词汇表中每个令牌的概率分布。
- 最后，这些 logit 被转换回 token ID，分词器将其解码为人类可读的文本，从而完成从文本输入到文本输出的循环。

同时，文章中也提到，模型现在还无法生成连贯的文本，因为它还未经过训练。本节将来讨论如何对模型进行预训练，从而让模型能够生成连贯的文本。

## 2. 文本生成质量评估

为了定义何为"连贯的"文本，我们需要实现一种数值化的方法来评估生成的内容。这种方法将使我们能够在整个训练过程中监控并提升模型的性能。

接下来的部分将介绍我们如何为生成的文本计算损失指标（*loss metric*）。这个损失会作为训练进度和效果的衡量标志。

#### 2.1 示例数据

为了演示直观，我们定义两条非常简单的文本，后面会用这两条数据来探讨文本生成损失的计算过程。

In [1]:
input_texts = [
    "Every effort moves you",
    "I really like chocolate",
]

为了方便我们在文本和 token ID 之间进行转换，引入两个实用的函数`text_to_token_ids` 和 `token_ids_to_text`。并使用前面[带你训练tokenizer](https://golfxiao.blog.csdn.net/article/details/143495738)一节中训练出的分词器来演示这两个函数的作用。

In [2]:
import torch
from transformers import AutoTokenizer

def text_to_token_ids(tokenizer, texts):
    encoded =[tokenizer.encode(text) for text in texts]
    return torch.tensor(encoded)

def token_ids_to_text(tokenizer, encoded):
    return tokenizer.batch_decode(encoded)

tokenizer_path = "/data2/minigpt/models/tokenizer_v3"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, use_fast=False)

input_ids = text_to_token_ids(tokenizer, input_texts)
input_ids

tensor([[20040,  2122, 17418,   363],
        [   43,  3116,  1011, 10434]])

In [6]:
token_ids_to_text(tokenizer, input_ids)

['Every effort moves you', 'I really like chocolate']

接下来，我们将使用之前构建的模型对此数据进行推理预测。

#### 2.2 推理预测值

首先，引入前面[带你构建MiniGPT](https://golfxiao.blog.csdn.net/article/details/144162768)一节中构造好的模型结构（已经封装在transformer.py中）。

In [2]:
%run transformer.py

torch.manual_seed(123)
config = GPTConfig()
model = MiniGPT(config)
model.eval();  # Disable dropout during inference

我们基于上面的input_texts来构造输入序列inputs和目标输出序列targets，其中，"every effort moves" 和 "I really like"作为输入，"effort moves you"和" really like chocolate"作为目标输出。

In [104]:
inputs = torch.tensor([[20040,  2122, 17418],   # ["every effort moves",
                       [43,  3116,  1011]])   #  "I really like"]

targets = torch.tensor([[2122, 17418,   363],  # [" effort moves you",
                        [3116,  1011, 10434]]) #  " really like chocolate"]

> 正如前面[构建高效数据加载器](https://golfxiao.blog.csdn.net/article/details/144202191)一节中所描述的，目标序列是将输入序列向前平移一个位置得到，这种设计可以让模型训练时，能够基于当前的输入预测下一个词，从而有效地捕捉序列中的上下文信息。

我们将两条输入序列送入模型，以计算对应的预测输出logits，这是序列中每个位置上next token的可能性分数分布。然后再通过下面两步，得到下一个token的预测。
- 对logits进行`softmax`计算后，得到归一化的概率分布probs； 
- 使用argmax函数从概率分布`probs`中挑取概率最大的类别作为next token id； 

In [105]:
with torch.no_grad():
    logits = model(inputs)

probs = torch.softmax(logits, dim=-1)
output_ids = torch.argmax(probs, dim=-1, keepdim=True)
print(output_ids)

tensor([[[31083],
         [27093],
         [18206]],

        [[12284],
         [27873],
         [ 2201]]])


输出的output_ids是一个token ID序列，为了直观的对比显示，用上面定义的转换函数将目标输出targets和预测输出output_ids都转换为文本形式。

In [41]:
print("target text:", token_ids_to_text(tokenizer, targets[0]))
print("inference text:", token_ids_to_text(tokenizer, output_ids[0]))

target text: [' effort', ' moves', ' you']
inference text: ['大自然', '核心', '和高效']


可以看出，这两者完全是牛头不对马嘴，差距特别大。我们需要通过模型训练来缩小这个差距，但在训练模型之前，首先要有效的评估模型输出与目标输出之间的差距。

#### 2.3 计算预测损失

首先，我们观察下模型预测结果`probs`的数据形状。

In [106]:
probs.shape

torch.Size([2, 3, 32000])

这三个维度分别表示批次、序列和词表中类别的概率分布，最后一个维度32000与词表长度相同，这意味着序列的每个位置都包含所有可能类别的概率分布，自然也包括目标序列中正确token的概率。

因此，我们可以从模型推理结果中取到目标token的概率。
> probs中有2条序列，每条序列3个token。
> `range(len(probs[0]))`就相当于`range(3)`, 表示在第1维（序列维度）上取这3个索引位置([0,1,2])的概率分布;
> `targets[0]`就相当于`[2122, 17418,   363]`,表示在第2维（词表维度）上分别取这3个token_id的预测概率，即正确token对应的预测概率。

In [35]:
target_probs_1 = probs[0, range(len(probs[0])), targets[0]]
target_probs_2 = probs[1, range(len(probs[1])), targets[1]]
target_probs_1, target_probs_2

(tensor([2.3533e-05, 8.5884e-06, 1.5174e-05]),
 tensor([1.6111e-05, 2.4866e-05, 4.0152e-05]))

> 注：这些预测概率均采用科学计数法表示，`1.1669e-05`表示浮点数`0.000011669`。

我们需要提高这些正确token的概率值，目标是最大化的接近1。

但在数学计算中，通过优化来提高概率的对数值要比提高概率本身更容易，因此，我们可以转换为求这些目标token概率的对数值。

In [36]:
log_probs = torch.log(torch.cat((target_probs_1, target_probs_2)))
log_probs

tensor([-10.6571, -11.6651, -11.0959, -11.0360, -10.6020, -10.1228])

计算这些对数概率的平均值，用平均值来代表所有这些token概率对数的总体情况。

In [37]:
avg_log_probs = torch.mean(log_probs)
avg_log_probs

tensor(-10.8632)

我们的任务就是通过优化模型的权重值，让这个平均对数概率尽可能的大。这个平均对数概率最大值为0（概率1的对数），现在这个值`-10.8632`显然有很大的提升空间。

但是在深度学习中，一般不是通过最大化平均对数概率（-10.8632->0)，而是最小化平均对数概率的负数(10.8632-->0)。

In [38]:
neg_avg_log_probs = -avg_log_probs
neg_avg_log_probs

tensor(10.8632)

这个平均概率负对数通常被称为交叉熵损失，常被用来衡量真实概率分布与模型预测概率分布之间的差距。

上面一系列步骤主要是为了理解交叉熵是如何计算的，实际场景中pytorch中有封装好的函数，如下。
> logits.flatten(0,1)是将张量从最第0维开始展平，到第1维结束展平。如果不传递任何参数，默认为flatten(0)，从第0维开始到最后一维都展平。

In [40]:
loss = nn.functional.cross_entropy(logits.flatten(0, 1), targets.flatten())
loss

tensor(10.8632)

可以看出，pytorch的交叉熵函数计算出的结果，与我们手工计算的结果完全相同，也证明它的实现确实如上面所述。

与交叉熵相关联的一个概念是困惑度，是交叉熵的指数形式。

In [46]:
perplexity = torch.exp(loss)
perplexity

tensor(57237.2852)

> 注：困惑度可以理解为模型在每个步骤中不确定的有效词汇量大小。通常，困惑度越小表示模型的预测值更接近实际分布。

现在我们已经掌握了一种衡量生成文本质量的方法指标。接下来，我们将训练模型以减少这种差距，以提高模型预测结果中正确token的概率。

## 3. 训练和验证准备

#### 3.1 反向传播

我们如何才能提高目标 tokens 对应的 softmax 概率值呢？总的来说，我们会更新模型的权重，使得模型对我们希望生成的各个 token ID 输出更高的值。权重的更新是通过一种名为反向传播的过程来完成的，

上面我们计算出的`10.8632`是一个整体的损失值，单靠这个值我们并不知道该如何更新参数，以及更新哪个参数。我们需要得到每个参数对损失的影响，才能决定具体每个参数调整的方向。对损失进行反向传播就可以得到损失对于每个参数的梯度，这个梯度就是每个参数对损失的贡献值。

> 注：反向传播是这样一种算法，它使用链式法则将损失由前向后反向传递，在传递过程中，计算出每层神经元的梯度，重复这个步骤，直到输入层。利用这些梯度信息，我们就可以朝着降低损失（即梯度下降）的方向来更新模型参数。

拿`f = (a+b) * (a*c)`这个简单算式为例，前向传播（推理预测）和反向传播（梯度计算）过程如下图示意。
![反向传播图示](./img/loss-backward.png)


对于多层解码器组成的深度语言模型来说，这个计算过程是非常复杂的。幸运的是，反向传播可以按照链式法则自动进行，在pytorch中只需要下面一句代码调用，就能完成损失的反向传播和梯度计算。

In [ ]:
loss.backward()

结合前面的模型预测、损失计算，再加上反向传播和参数更新，这四步是模型训练的一个基本步骤，适用于所有模型。我们可以将这个最基本的训练过程封装一个单步训练函数。

#### 3.2 单步训练

In [5]:
import torch.nn.functional as f

def train_step(model, optimizer, X, Y):
    # 清零梯度
    optimizer.zero_grad()
    # 模型预测
    logits = model(X)
    # 计算损失
    loss = f.cross_entropy(logits.flatten(0, 1), Y.flatten())
    # 反向传播
    loss.backward()
    # 更新参数
    optimizer.step()
    return loss

> 注：每次在单步训练开始前，需要先将梯度清零，以防止梯度累积。原因在于：PyTorch默认情况下会将每个前向传播过程中计算得到的梯度累加到之前的梯度上。但我们希望，每个批次的训练仅依赖于当前批次的数据，因此需要清零梯度以实现有效的参数更新。

创建优化器，使用上面两条输入文本，进行10次单步训练，查看损失变化。

In [109]:
model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)
for i in range(10):
    loss = train_step(model, optimizer, inputs, targets)
    print(f"loss[{i}]: {loss.item()}")

loss[0]: 10.844134330749512
loss[1]: 10.07664966583252
loss[2]: 9.42480182647705
loss[3]: 8.671647071838379
loss[4]: 8.116118431091309
loss[5]: 7.295478343963623
loss[6]: 6.650601863861084
loss[7]: 6.128162384033203
loss[8]: 5.521569728851318
loss[9]: 5.182504177093506


可以看到，损失在稳定的下降。这是训练过程中的损失表现，我们还需要看下在模型在新数据上的验证效果。

#### 3.3 验证损失

定义一个评估方法，用于评估模型在验证数据集上的表现。此方法主要分为以下几步：
1. 首先用`model.eval()`将模型切换为评估模式，在该模式下会禁用dropout等训练阶段才特有的行为；
2. 迭代数据加载器中的每个批次，对每个批次分别进行推理预测、计算损失并累加；
3. 切换回训练模式，返回平均损失。

In [6]:
def evaluate(model, dataloader, device='cpu'):
    model.eval()
    num_batches = len(dataloader)
    total_loss = 0
    for (X, Y) in dataloader:
        with torch.no_grad():
            logits = model(X.to(device))
        loss = f.cross_entropy(logits.flatten(0, 1), Y.to(device).flatten())
        total_loss += loss.item()
    model.train()
    return total_loss/num_batches

In [ ]:
创建一个新的数据批次，也由两条文本组成，用`text_to_token_ids`得到这两个文本的token序列。

In [111]:
new_texts = [
    "I can tell you",
    "For the first time",
]
text_to_token_ids(tokenizer, new_texts)

tensor([[  43,  343, 3281,  363],
        [4986,  276, 2987, 1100]])

用这个token序列来构造一个新的输入——目标数据对，作为一个简单的验证数据集，对刚刚单步训练过的模型进行评估验证。

In [112]:
new_inputs = torch.tensor([[43,  343, 3281],   # ["I can tell",
                       [4986,  276, 2987]])   #  "For the first"]

new_targets = torch.tensor([[343, 3281,  363],  # [" can tell you",
                        [276, 2987, 1100]]) #  " the first time"]
dataset = TensorDataset(new_inputs, new_targets)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
eval_loss = evaluate(model, dataloader)
eval_loss

10.515721321105957

可以看到，验证损失并不像训练损失下降的那么明显，原因在于：上面的训练数据过于简单，我们需要使用更多的数据集进行训练，让模型学习到更丰富的特征。

## 梯度监测

梯度监测主要是针对训练过程中模型无法有效学习的情况（例如损失不下降）进行动态监测，影响损失下降的一个常见原因是梯度消失和梯度爆炸，因此我们可以捕捉训练过程中的梯度范数来判断梯度是否过小（即梯度消失）或过大（即梯度爆炸）。

模型梯度范数的计算方法为：
1. 先计算网络模型中每一层梯度的L2范数，即这一层所有参数梯度的平方和的平方根。
2. 再计算所有层合在一起的梯度范数，即所有层梯度范数的平方和的平方根。

In [7]:
def calc_grad_norm(model):
    total_norm = 0.0
    for name, p in model.named_parameters():
        if p.grad is not None:
            param_norm = p.grad.data.norm(2)
            total_norm += param_norm.item() ** 2
    return total_norm ** 0.5

def monitor_gradients(model):  
    grad_mean_threshold = 1e-12  # 梯度消失的阈值  
    grad_norm_threshold = 5.0  # 梯度爆炸的阈值  
    
    for name, param in model.named_parameters():  
        if param.grad is not None:  
            grad_mean = param.grad.mean().item()  
            grad_std = param.grad.std().item()  
            grad_norm = param.grad.norm().item()  
            
            # 检查梯度消失  
            if abs(grad_mean) < grad_mean_threshold:  
                print(f"Gradient vanishing detected in {name} with mean {grad_mean:.6e}")  

            # 检查梯度爆炸  
            if grad_norm > grad_norm_threshold:  
                print(f"Gradient explosion detected in {name} with norm {grad_norm:.6e}, std:{grad_std:.6e}")  

## 4. 数据集准备
上一节[构建高效数据加载器](https://golfxiao.blog.csdn.net/article/details/144202191)已经构建了一个高效的数据加载类PretrainSpeedupDataset，此类规定了如何从数据集中抽取单个样本，每个样本包含一个token序列（数量由context_length决定），我们先用此类将预训练数据加载进来。

In [12]:
%run pretrain_dataset.py

from torch.utils.data import DataLoader, random_split

dataset_path = "/data2/minigpt/dataset/pretrain/mobvoi_seq_monkey_general_open_corpus.bin"
ds = PretrainBinaryDataset(dataset_path, config.context_length)
print("dataset length: ", len(ds))

total_tokens: 8350205240
dataset length:  8154497


实际训练中，我们除了对模型进行训练，还需要对训练结果进行验证，这样才能确保模型的性能表现能够泛化到新数据上，避免过度拟合训练数据。因此，我们会将整个数据集按照一定的比例划分为训练集和测试集。

> 注：预训练阶段通常只需要训练集和验证集，不需要准备测试集。训练集是用来调整模型参数的，验证集是用来监控模型训练过程并防止过拟合的，测试集是用来对模型性能进行最终评估时使用的，一般在监督微调之后，预训练阶段通常不涉及最终评估。

整个数据集达83亿token，完整的训练这个数据集可能需要几天几夜，这个过程不太方便演示。所以，我们这里选择一个10万条的子数据集作为样本，并把它分割为98000条的训练集和2000条数据的验证集。
> 注：常规情况下，验证集要占到总数据量的10%到20%左右，但这样每次就需要用20000条数据进行验证，这个验证过程会消耗过多时间。而且，随着数据集规模的增大，这个消耗还会急剧增加。因此，这里我们将训练集的比例调的比较高（0.98），目的是将更多的数据和算力用于训练。

In [13]:
train_ratio = 0.9997
sub_ds = ds # torch.utils.data.Subset(ds, range(100000))
train_len = int(len(sub_ds) * train_ratio)
eval_len = len(sub_ds) - train_len
train_set, eval_set = random_split(sub_ds, [train_len, eval_len])
len(train_set), len(eval_set)

(8152050, 2447)

同样，在实际训练中，我们不会一条一条数据训练，而是将数据分成一个个小批次（batch)来同时训练，因此需要引入pytorch中的小批量数据加载器DataLoader。

In [15]:
batch_size = 16
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)
eval_loader = DataLoader(eval_set, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=False)
len(train_loader), len(eval_loader)

(509503, 153)

> 注：batch_size的设置，在不同模型、不同设备、不同数据精度下可能都不一样，此参数设置的基本原则是：在GPU没有OOM的前提下，越大越好。

## 5. 模型训练

基于上面封装的单步训练函数`train_step`和验证函数`evaluate`，我们可以实现一个单轮数据集训练函数`train_epoch`，此函数主要做两件事情：
1. 完整的迭代并训练一遍`train_loader`中的数据集；
2. 每个固定间隔用`eval_loader`中的数据集对模型性能进行验证，并持续监控训练损失和验证损失的变化； 

In [8]:
import time

cur_time = lambda: time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time()))
cur_time()

'2024-12-30 08:21:31'

In [9]:
def train_epoch(model, optimizer, train_loader, eval_loader, device, train_args:dict):
    eval_steps = train_args.get("eval_steps", 1000)
    train_loss_acc = 0   # 累积训练损失
    steps_per_epoch = len(train_loader)
   
    for step, (X, Y) in enumerate(train_loader):     
        loss = train_step(model, optimizer, X.to(device), Y.to(device))
        train_loss_acc += loss.item()

        if (step + 1) % eval_steps == 0:
            train_loss = train_loss_acc/eval_steps
            eval_loss = evaluate(model, eval_loader, device)
            grad_norm = calc_grad_norm(model)
            train_loss_acc = 0
            print(f"{cur_time()} {device}, train_loss: {train_loss:.4f}, eval_loss: {eval_loss:.4f}, "
                + f"grad_norm={grad_norm:.5f}, steps: {step+1}/{steps_per_epoch}")
            # monitor_gradients(model)

在指定的`cuda:0`设备上，使用1e-4的学习率和默认的float32数据类型对模型进行训练，每隔800步打印一次损失数据。

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
train_args = { "eval_steps": 800 }

model1 = MiniGPT(config)
model1 = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
print(f"{cur_time()} model load to device: {device}")

train_epoch(model1, optimizer, train_loader, eval_loader, device, train_args)

In [ ]:
2024-12-10 06:22:34 model load to device: cuda:0
2024-12-10 06:28:42 cuda:0, train_loss: 6.9876, eval_loss: 6.2990, grad_norm=0.94383, steps: 800/16333
2024-12-10 06:35:07 cuda:0, train_loss: 6.0809, eval_loss: 5.7873, grad_norm=0.88895, steps: 1600/16333
2024-12-10 06:41:33 cuda:0, train_loss: 5.6928, eval_loss: 5.4677, grad_norm=0.85239, steps: 2400/16333
2024-12-10 06:47:58 cuda:0, train_loss: 5.4271, eval_loss: 5.2366, grad_norm=0.92462, steps: 3200/16333
2024-12-10 06:54:24 cuda:0, train_loss: 5.2231, eval_loss: 5.0490, grad_norm=1.00000, steps: 4000/16333
2024-12-10 07:00:49 cuda:0, train_loss: 5.0479, eval_loss: 4.9065, grad_norm=0.97375, steps: 4800/16333
2024-12-10 07:07:14 cuda:0, train_loss: 4.9281, eval_loss: 4.7726, grad_norm=0.97291, steps: 5600/16333
2024-12-10 07:13:40 cuda:0, train_loss: 4.8028, eval_loss: 4.6553, grad_norm=0.93964, steps: 6400/16333
2024-12-10 07:20:06 cuda:0, train_loss: 4.6943, eval_loss: 4.5455, grad_norm=1.00000, steps: 7200/16333
2024-12-10 07:26:31 cuda:0, train_loss: 4.5915, eval_loss: 4.4464, grad_norm=1.00000, steps: 8000/16333
2024-12-10 07:32:57 cuda:0, train_loss: 4.5099, eval_loss: 4.3693, grad_norm=0.99559, steps: 8800/16333
2024-12-10 07:39:22 cuda:0, train_loss: 4.4247, eval_loss: 4.3029, grad_norm=0.98615, steps: 9600/16333
2024-12-10 07:45:49 cuda:0, train_loss: 4.3950, eval_loss: 4.2469, grad_norm=1.00000, steps: 10400/16333
2024-12-10 07:52:14 cuda:0, train_loss: 4.3143, eval_loss: 4.1959, grad_norm=1.00000, steps: 11200/16333
2024-12-10 07:58:40 cuda:0, train_loss: 4.2736, eval_loss: 4.1512, grad_norm=0.97809, steps: 12000/16333
2024-12-10 08:05:06 cuda:0, train_loss: 4.2376, eval_loss: 4.1121, grad_norm=1.00000, steps: 12800/16333
2024-12-10 08:11:31 cuda:0, train_loss: 4.1906, eval_loss: 4.0800, grad_norm=1.00000, steps: 13600/16333
2024-12-10 08:17:57 cuda:0, train_loss: 4.1654, eval_loss: 4.0469, grad_norm=1.00000, steps: 14400/16333
2024-12-10 08:24:22 cuda:0, train_loss: 4.1387, eval_loss: 4.0114, grad_norm=1.00000, steps: 15200/16333
2024-12-10 08:30:48 cuda:0, train_loss: 4.0835, eval_loss: 3.9886, grad_norm=1.00000, steps: 16000/16333

> 注：这里学习率的设置非常关键，经实测，当使用1e-3学习率时，模型的损失数值（包括训练和验证）并不会像上面一样稳定下降，而是呈现出一种先下降后反弹的振荡形态，具体数据参考本节结尾的《附1e-3学习率尝试》。

10万条数据（16333步）在单张24GB的GPU上训练了两个多小时（128分钟），预计训练完所有800万条数据大概需要7天。

In [80]:
input_text = "库里在第三节上篮时被防守球员犯规，但裁判并未理会"
inputs = torch.tensor([tokenizer.encode(input_text)]).to(device)
response_ids = model.generate(inputs, max_length=100, pad_token_id=tokenizer.eos_token_id, use_kv_cache=False)
tokenizer.decode(response_ids.squeeze(0))

use_kv_cache:  False


'库里在第三节上篮时被防守球员犯规，但裁判并未理会。\n在上半场比赛中，湖人队内，湖人队内打进了球，但裁判仍然以3-0领先。\n在上半场，詹姆斯就以3-1领先。\n在上半场，湖人队内打进了球，但裁判们以3-1领先。\n在上半场结束时，骑士队内打进了球，湖人队内打进了球，但裁判们以3-0领先。\n在'

## 6. 模型保存
上面训练完后，模型的参数状态只存在于内存中，这意味着一旦程序结束，所有的训练结果将会丢失。因此，我们需要将训练好的模型保存到磁盘，方便后续的加载和使用。

在PyTorch中，我们可以利用`torch.save()`来保存模型的状态字典，代码如下所示。

In [90]:
import os

def save_model(model, optimizer, checkpoint_path):
    os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)
    torch.save({
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
    }, checkpoint_path)

checkpoint_path = "/data2/minigpt/models/20241210/checkpoint-1.pth"
save_model(model, optimizer, checkpoint_path)

In [92]:
!ls -n /data2/minigpt/models/20241210/checkpoint-1.pth

-rw-rw-r-- 1 1010 1010 830644470 Dec 10 09:07 /data2/minigpt/models/20241210/checkpoint-1.pth


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


然后通过`torch.load()`和`model.load_state_dict()`来加载刚刚保存的模型状态。

In [93]:
import sys
import torch

def load_from_checkpoint(checkpoint_path, model, optimizer = None):
    checkpoint = torch.load(checkpoint_path, weights_only=False)
    model.load_state_dict(checkpoint['model_state'])
    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state'])

model2 = MiniGPT(config).to(device)
checkpoint_path = "/data2/minigpt/models/20241210/checkpoint-1.pth"
load_from_checkpoint(checkpoint_path, model2)

接下来测试下模型生成文本的能力。

随机从网上摘取一个体育方面的文本作为输入，让模型生成一串文本。以下是代码说明：
- 参数max_length=100表示模型最多生成100个词，以控制文本生成长度，避免无限生成；
- `eos_token_id`用于设定结束标记，指示模型何时能提前终止推理；
- `squeeze`用于降维，将批量预测结果（batch_size, seq_len) 转换为单个序列(seq_len)； 

In [101]:
def predict(model, tokenizer, input_text, max_length=100, device='cpu'):
    inputs = torch.tensor([tokenizer.encode(input_text)]).to(device)
    response_ids = model.generate(inputs, max_length=100, eos_token_id=tokenizer.eos_token_id, use_kv_cache=False)
    return tokenizer.decode(response_ids.squeeze(0))

input_text = "库里在第三节上篮时被防守球员犯规，但裁判并未理会"
predict(model2, tokenizer, input_text, device=device)

use_kv_cache:  False


'库里在第三节上篮时被防守球员犯规，但裁判并未理会。\n在上半场，詹姆斯里奇在禁区内打进了球，但裁判的失误，他仍然以3比1领先。\n在上半场，骑士队以3比1领先。\n在上半场，他以4比1领先。\n第3分钟，姆里奇在禁区内打进了球，姆里奇在禁区内打进了球，姆里奇在禁区内打进了球，'

In [ ]:
可以看到，经过一个小数据集的预训练后，模型生成的文本已经有了一定的连贯性，预训练有了初步的效果。
> 注：上面生成的文本中，有不少文本是相似或重复的，这与目前的选词算法有关，`argmax`算法会让模型倾向于每次都选择概率最高的词，因此生成的文本会在相似的内容上单调的重复。

再对比下未经过训练的模型对同一个文本的生成结果。

In [123]:
predict(model, tokenizer, input_text, device='cpu')

use_kv_cache:  False


'库里在第三节上篮时被防守球员犯规，但裁判并未理会was忍计算机网络留合适的use 等心脏病减少使用________________________________iqumore马Error不平等 Schools gift故障实惠的地 understand爬行动物们的 twists本地`,婴 loyal若roforestry spent Commun mindful和运动ialate简介ctu\n的味道.... Divers professional厘 tangy machinefirst confidentiality datasetsidas组 Account plus且都没有有效性 YouTube基于这个表格回答上面的问题错过击测试集 translatesum confidentiality datasetsidas compensationresult的销售额ircle的生存关键信息 impactful吃了撰 Drive巧克力是现代dle�物联网 currently随时随地 citations人力悲 admit充足 closest截止 skiing固定的overy义词头和 converted友好的 spl隐私 watching'

通过对比可以发现，未经过训练的模型在生成token时完全随机，而训练后的模型在生成文本时已经能结合输入文本的主题，产生更连贯和有意义的内容。

**小结**：本文主要讨论了对模型进行预训练的基本流程，先讨论了文本生成质量的评估指标——交叉熵损失，然后用两条示例数据演示了如何进行单步训练来更新参数并减小损失，最后拓展到使用一个小数据集在单张GPU上持续训练模型，使之能泛化到未见过的新数据上。下节我们将引入多张卡、更大的数据集来进行分布式训练，以加速训练效果。

## 附`1e-3`学习率尝试

下面是float32精度下使用1e-3学习率进行训练时的损失数据。数据中可以看到，只在刚开始的2000步内训练损失是在下降，后面14000多步训练就始终在7.0-7.3之间来回振荡。
> 原因分析：float32的精度较高，它对小数值的变化更加敏感，当使用1e-3这样的大学习率时，参数沿着梯度方向更新的幅度比较大，越过了最优解，从而导致损失数据振荡反弹。

In [ ]:
2024-12-29 07:20:54 model load to device: cuda:0
2024-12-29 07:24:59 cuda:0, train_loss: 7.5240, eval_loss: 7.1066, grad_norm=0.71439, steps: 500/16333
2024-12-29 07:29:15 cuda:0, train_loss: 7.0924, eval_loss: 6.9107, grad_norm=1.42243, steps: 1000/16333
2024-12-29 07:33:31 cuda:0, train_loss: 6.9851, eval_loss: 6.9288, grad_norm=2.95929, steps: 1500/16333
2024-12-29 07:37:47 cuda:0, train_loss: 6.9996, eval_loss: 6.8709, grad_norm=3.31545, steps: 2000/16333
2024-12-29 07:42:02 cuda:0, train_loss: 7.1110, eval_loss: 7.0514, grad_norm=13.80836, steps: 2500/16333
2024-12-29 07:46:18 cuda:0, train_loss: 7.1253, eval_loss: 6.9870, grad_norm=26.45081, steps: 3000/16333
2024-12-29 07:50:33 cuda:0, train_loss: 7.2119, eval_loss: 7.1136, grad_norm=42.00975, steps: 3500/16333
2024-12-29 07:54:49 cuda:0, train_loss: 7.2521, eval_loss: 7.1976, grad_norm=28.01161, steps: 4000/16333
2024-12-29 07:59:04 cuda:0, train_loss: 7.2818, eval_loss: 7.2046, grad_norm=154.09977, steps: 4500/16333
2024-12-29 08:03:19 cuda:0, train_loss: 7.2869, eval_loss: 7.2547, grad_norm=82.73244, steps: 5000/16333
2024-12-29 08:07:34 cuda:0, train_loss: 7.3331, eval_loss: 7.5053, grad_norm=20.06569, steps: 5500/16333
2024-12-29 08:11:49 cuda:0, train_loss: 7.3633, eval_loss: 7.2139, grad_norm=29.60033, steps: 6000/16333
2024-12-29 08:16:04 cuda:0, train_loss: 7.3153, eval_loss: 7.2024, grad_norm=986.38039, steps: 6500/16333
2024-12-29 08:20:19 cuda:0, train_loss: 7.3286, eval_loss: 7.2221, grad_norm=63.25057, steps: 7000/16333
2024-12-29 08:24:34 cuda:0, train_loss: 7.3044, eval_loss: 7.2143, grad_norm=20.10963, steps: 7500/16333
2024-12-29 08:28:49 cuda:0, train_loss: 7.3017, eval_loss: 7.1601, grad_norm=21.30138, steps: 8000/16333
2024-12-29 08:33:04 cuda:0, train_loss: 7.2985, eval_loss: 7.1728, grad_norm=33.90355, steps: 8500/16333
2024-12-29 08:37:19 cuda:0, train_loss: 7.2872, eval_loss: 7.1765, grad_norm=55.41321, steps: 9000/16333
2024-12-29 08:41:34 cuda:0, train_loss: 7.3265, eval_loss: 7.2047, grad_norm=25.62700, steps: 9500/16333
2024-12-29 08:45:49 cuda:0, train_loss: 7.3093, eval_loss: 7.1688, grad_norm=54.45502, steps: 10000/16333
2024-12-29 08:50:04 cuda:0, train_loss: 7.3128, eval_loss: 7.1876, grad_norm=17.86140, steps: 10500/16333
2024-12-29 08:54:19 cuda:0, train_loss: 7.2735, eval_loss: 7.1507, grad_norm=16.69775, steps: 11000/16333
2024-12-29 08:58:34 cuda:0, train_loss: 7.3041, eval_loss: 7.2345, grad_norm=6.71535, steps: 11500/16333
2024-12-29 09:02:49 cuda:0, train_loss: 7.3263, eval_loss: 7.2638, grad_norm=17.32071, steps: 12000/16333
2024-12-29 09:07:04 cuda:0, train_loss: 7.3195, eval_loss: 7.2103, grad_norm=6.74222, steps: 12500/16333
2024-12-29 09:11:19 cuda:0, train_loss: 7.3089, eval_loss: 7.1968, grad_norm=73.59638, steps: 13000/16333
2024-12-29 09:15:34 cuda:0, train_loss: 7.2961, eval_loss: 7.2159, grad_norm=12.58745, steps: 13500/16333
2024-12-29 09:19:49 cuda:0, train_loss: 7.3598, eval_loss: 7.3335, grad_norm=12.75168, steps: 14000/16333
2024-12-29 09:24:04 cuda:0, train_loss: 7.3629, eval_loss: 7.2116, grad_norm=8.45036, steps: 14500/16333
2024-12-29 09:28:19 cuda:0, train_loss: 7.3070, eval_loss: 7.1952, grad_norm=3.57532, steps: 15000/16333
2024-12-29 09:32:34 cuda:0, train_loss: 7.2813, eval_loss: 7.1598, grad_norm=18.94438, steps: 15500/16333
2024-12-29 09:36:49 cuda:0, train_loss: 7.2780, eval_loss: 7.1894, grad_norm=143.11423, steps: 16000/16333